# **Lab V : Identification des paramètres d'une sinusoïde (suite)**

+ **Cours "Physique du Numérique"** - Portail René Descartes - AMU

Préparé par :

- Jean-Marc Themlin (v. 2021-09), Aix-Marseille Université © Contenus à diffusion restreinte, dans le cadre de ce cours.

------------------

### Remarque
A toutes fins utiles, on rappelle ici qu'une solution de secours (moins pratique, et probablement moins stable) permet d'accéder aux énoncés sur des notebooks fonctionnels sur un serveur distant (Binder) uniquement depuis votre navigateur. Vous pourrez exécuter, modifier et compléter les cellules de code et remplir les cellules de texte ; à l'issue de votre travail, vous pourrez importer (commande `Download` dans le menu *File*) le notebook complété pour le stocker localement sur votre PC et le déposer sur AMeTICE. Pour accéder à Binder pour travailler ce premier TP, il suffit de cliquer sur l'icône ci-dessous et de sélectionner dans le répertoire des fichiers accessibles, le notebook *Enonce_Lab_4_Identification_Sinusoides.ipynb* ; soyez patient, ça peut prendre quelques minutes pour lancer l'environnement :

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Pango01/conda/HEAD/)

-------------

## Objectif du TP V

Dans le TP IV, vous avez conçu et réalisé un programme capable d’estimer les paramètres A et $\phi$ d’un signal sinusoïdal de fréquence $f_0$ connue. Vous allez maintenant compléter le processus en réalisant un programme capable d’estimer l’ensemble des paramètres (A, $f_0$, $\phi$) d’un signal sinusoïdal dont même la fréquence est inconnue au départ.

## Principe de la méthode

On se reportera aux explications données au cours, dont voici le résumé. Pour la détermination de la fréquence $f_0$, le principe consiste à comparer la sinusoïde d’entrée avec plusieurs signaux de référence, des sinusoïdes « d’essai » (en fait des exponentielles complexes) dont les fréquences $f_k$ couvrent une gamme suffisamment large, de 0 à la fréquence maximale acceptable, donnée par la fréquence de Nyquist $f_{max}=\frac{f_e}{2}$). Sur la base d’un produit scalaire, la sinusoïde de fréquence $f_k$ qui ressemblera le plus au signal d’entrée (celle dont le produit scalaire avec le signal d'entrée aura le module le plus élevé) vous permettra d’estimer $f_0 = f_k$.

<img src="https://amubox.univ-amu.fr/s/QAAZYkJMnwerL3W/preview" style="zoom:75%;" />


### Programmation : Quelques fonctions Python utiles

Exécutez `Help(nom_de_la_fonction)` ou `nom_de_la_fonction?` pour afficher l'aide des fonctions ci-dessous et comprendre ce qu'elles réalisent.

**Dans la bibliothèque *numpy* :** *np.min(x)* ou *np.max(x)* retournent la valeur du minimum ou du maximum du vecteur x. Une instruction du type *np.where(x==max(x))* permet de retrouver l'indice (*i.e.* la position du maximum de *x*).     

**Exercice :** Examinez le script de la cellule ci-dessous pour bien comprendre comment mettre en oeuvre ces instructions, et complétez le script pour afficher le graphe de y en fonction de t. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Te=1/10
n = np.arange(0,2**8)
t = n*Te
y = (x-128)**2-4*(x-128)-1000
jack=np.where(y==np.min(y))
print(jack,jack[0],y[jack])

print("Le minimum de y se trouve à l'indice n = ",jack[0])
print("Le minimum de y se trouve en t = ",t[jack[0]])
print("Le minimum de y vaut y[min] = ",y[jack[0]])

# A compléter pour afficher le graphe de y en fonction de t



## **Problème V-1** : Estimer la fréquence, l'amplitude et la phase d'un signal sinusoïdal de fréquence inconnue

Utilisez la cellule de code ci-dessous pour construire une fonction Python *apfc* pour estimer la fréquence $f_{Est}$, l'amplitude $A_{Est}$ et la phase à l'origine $\phi_{Est}$ d'un signal (co)sinusoïdal discret $s[n]$ de fréquence inconnue. 

#### Les paramètres d'entrée de la fonction seront :

- le vecteur support temporel du signal
- le signal à analyser (dont vous allez estimer les paramètres)

#### Les paramètres retournés par la fonction seront :

- $f_{Est}$, la fréquence estimée de la sinusoïde
- $A_{Est}$, l'amplitude estimée de la sinusoïde
- $\phi_{Est}$, la phase à l'origine estimée de la sinusoïde
- Le résultat des produits scalaires entre le signal et l'ensemble des signaux de références 

<img src="https://amubox.univ-amu.fr/s/kbz86mdye3z9QT3/preview" style="zoom:50%;" />

Une bonne habitude en programmation est de bien réfléchir à ce que l’on veut réaliser et aux étapes du programme avant de se lancer « tête baissée » dans l’écriture du code. Pour vous aider à établir le script qui réalise l’estimation des paramètres, nous vous recommandons d’établir d’abord son **organigramme**.

Analysez les différentes étapes listées en ordre aléatoire dans le tableau ci-dessous et placez leur numéro dans le bon ordre pour reconstituer l’organigramme de la fonction, qui comprend notamment une boucle `for` : 

| Etapes |                          Opérations                          |
| :----: | :----------------------------------------------------------: |
|   1    | Calcul des paramètres estimés A et $\phi$ à l’aide de $PS_{Max}=PS(mm)$, <br />et affichage de ces valeurs A et $\phi$. |
|   2    | Définition d’un vecteur $f[k]$ contenant les N/2 fréquences d’essais $f_k=k/Dur$. |
|   3    |      Définition du signal sinusoïdal $s(t)$ à estimer.       |
|   4    | Remplissage d’un vecteur PS contenant les N/2 produits scalaires normalisés (par la durée) entre le signal à estimer et les signaux de référence $r_k[n]$. |
|   5    | Définition des signaux de référence du type $r_k[n]=e^{j2\pi f_k  n T_e}$ |
|   6    |        Calcul de la durée $Dur$ du signal à estimer.         |
|   7    |                   Boucle `for k=0 : N/2+1`                   |
|   8    | Détermination de la position mm (l’indice mm) de la valeur maximale <br />des produits scalaires contenus dans le vecteur PS. |
|   9    |       On en déduit la valeur estimée $f_{Est}=mm/Dur$.       |
|   10   | Superposition du signal initial $s(nTe)$ avec une sinusoïde reconstituée<br /> avec les paramètres estimés $f_{Est}$, $A_{Est}$ et $\phi_{Est}$. |


> **Q1**: Ecrivez ici dans le bon ordre les étapes de l'algorithme permettant d'estimer les trois paramètres d'un signal sinusoïdal.

> **Q1-R**: Votre réponse ici : 

### **V-1.1** : Premier test de la fonction $apfc$

Pour tester votre fonction, générez dans la cellule ci-dessous le signal de test sinusoïdal $s[n]$. Pour vérifier la validité de l'estimation, superposez dans un graphe la sinusoïde initiale et la (co)sinusoïde générée à partir des paramètres estimés numériquement (avec *apfc*).

> **Q2** : Remplissez la table ci-dessous avec les résultats de l'estimation numérique des paramètres.

|                          | $f_{Est}$ | $\phi_{Est}$ | $\phi_{Est}$ |
| :----------------------: | :-------: | :----------: |:------------:|
|  Estimation "manuelle"   |           |              |              |
| Estimation "automatique" |           |              |              |


In [1]:
import matplotlib.pyplot as plt
import numpy as np

t = np.arange(0,0.998,0.002)
s = 1.3**3*np.cos(2*np.pi*5*t+np.sqrt(7.84))

# A compléter

### **V-1.2** : Visualisation du module des produits scalaires (la TF discrète)

Dans la cellule ci-dessous, ajoutez les instructions pour tracer dans un graphe les différentes valeurs du module des produits scalaires (normalisés par la durée) $\left\langle {\widetilde{s,r}} \right\rangle \left[ {k } \right]$ en fonction des fréquences d’essai $f_k$. 

> **Q3**: A la vue de ce graphe, pouvez-vous confirmer que l'hypothèse qui postule que les produits scalaires de tests tendent pour la plupart vers zéro, à part celui correspondant à la fréquence d'essai $f_k=f_0$ ? 

> **Q3-R**: Votre réponse ici : 

##### Le graphe que vous venez de tracer représente le **module de la transformée de Fourier discrète (TFD)** du signal numérique s[n].

----------------------------------------


### **V-1.3** : Test d'*apfc* sur un signal de paramètres inconnus

Vous allez à présent tester le bon fonctionnement de votre fonction *apfc* sur un signal de paramètres inconnus. Les premières instructions de la cellule ci-dessous vont télécharger dans l'environnement le fichier *lab_data.mat*, qui contient les variables suivantes :

- le vecteur support temporel *t_samp*
- le vecteur signal *s_samp*, qui contient les échantillons d'une sinusoïde discrète d'une fréquence $f_0=100 \ Hz$.

Les valeurs retournées par votre script seront les suivants :

- la période d'échantillonnage $\Delta t$ 
- le nombre total $N$ de points que contiennent chacun de ces deux vecteurs
- la valeur de la fréquence estimée $f_{Est}$
- la valeur de l'amplitude estimée $A_{Est}$
- la valeur de la phase à l'origine estimée $\phi_{Est}$ 

Pour vérifier la validité de l'estimation, complétez la cellule de code ci-dessous avec les instructions Python permettant de superposer dans un graphe la sinusoïde initiale (*s_samp* en fonction de *t_samp*) avec la (co)sinusoïde générée à partir des paramètres estimés numériquement. 

In [ ]:
#
#  Imports from a remote .mat (MatLab) file
#
import requests
from scipy.io import loadmat

git_path="https://raw.githubusercontent.com/Pango01/Signal00/main/"
r=requests.get(git_path+'lab_data_apc.mat')
print(r.status_code)    #  returns 200 if opening is OK

with open('mytext.mat', 'wb') as f:
    f.write(r.content)
data=loadmat('mytext.mat')

tt = data['t_samp']
ss = data['s_samp']
t_samp = tt[0,]
s_samp = ss[0,]
tt = data['t_sonar']
ss = data['s_sonar']
t_sonar = tt[0,]
s_sonar = ss[0,]

print("Vérification t_samp et s_samp")
print(t_samp.ndim, t_samp.shape, t_samp.size, t[0])
print(s_samp.ndim, t_samp.shape, s_samp.size, s[0])
print("Vérification t_sonar et s_sonar")
print(t_sonar.ndim, t_sonar.shape, t_sonar.size, t[0])
print(s_sonar.ndim, t_sonar.shape, s_sonar.size, s[0])

# A compléter avec l'appel à apfc et la construction du graphe


-------------------------------------------

## **Problème V-2** : Utilisation des produits scalaires pour estimer le spectre unilatéral des signaux

Les produits scalaires calculés par la fonction *apfc* représentent la transformée de Fourier discrète (TFD) du signal numérique s[n]. Bien utilisé, la TFD permet d'évaluer numériquement le spectre (uni- ou bilatéral) d'un signal numérique. 

Utilisez la cellule ci-dessous en appelant `apfc` pour estimer les paramètres du signal numérique w[t] suivant, où vous pouvez garder le vecteur support temporel t défini plus haut :

$w[t]=2cos(2\pi t)+4cos(2\pi3t-\frac{\pi}{4})+2cos(2\pi5t)$

> **Q4**: A quoi correspondent les valeurs estimées fournies par `apfc` ? 

> **Q4-R**: Votre réponse ici : 


Tracez dans une figure à deux graphes les modules et des phases des produits scalaires (normalisés par la durée)  $\left\langle {\widetilde{s,r}} \right\rangle \left[ {k } \right]$ en fonction des fréquences $f_k$. 

> **Q5**: Quelles informations supplémentaires pouvez-vous déduire de ce dernier graphe ? 

> **Q5-R**: Votre réponse ici : 



## **Problème V-3** : Application : Estimation de la vitesse d'un mobile via le décalage Doppler d'un signal sonore (SONAR)

<img src="https://dosits.org/wp-content/uploads/2017/07/anime66.gif"  class="center">

Un sonar doppler émet dans l’océan un signal sonore sinusoïdal d’une fréquence $f_e=1 kHz$. Ce signal est réfléchi par un objet mobile et donne lieu au signal reçu `s_sonar` sur un support temporel `t_sonar`, disponibles dans votre environnement. 

La vitesse du son dans l’eau salée $v_{son}$ étant approximativement de 1450 m/s, utilisez votre fonction apfc pour estimer la fréquence du signal sinusoïdal réfléchi, ainsi que la vitesse de l’objet mobile. 

On rappelle que la fréquence $f_r$ de l'onde sonore réfléchie est liée à la fréquence de l'onde émise, à la vitesse de propagation de l'onde sonore dans l'eau salée $v_{son}$, et à la vitesse du mobile $v_M$ ($v_M$ positif si le mobile s'approche du récepteur, et négatif s'il s'en éloigne), par la formule suivante (effet Doppler-Fizeau) :

$$
f_r = f_e {v_{son} \over {v_{son} - v_M }}
$$

**Remarque :** le vecteur `s_sonar` étant très long, le temps de calcul peut l’être aussi : soyez patients ! Indiquez dans votre rapport le temps pris par votre fonction apfc pour calculer les paramètres estimés. 


> **Q6**: Quelle est la fréquence estimée de la sinusoïde réfléchie par l'objet mobile ? 

> **Q6-R**: Votre réponse ici : 


> **Q7**: Quelle est la vitesse estimée de l'objet mobile détecté par le sonar-Doppler ? 

> **Q7-R**: Votre réponse ici :

------

## Conclusions personnelles

Indiquez ci-dessous le temps approximatif que vous passé à travailler ce TP en-dehors de la séance.

> **Q8-R** (votre réponse ici) :

Ecrivez ci-dessous, en guise de conclusions, quelques phrases décrivant ce que ce TP vous a appris.

> **Q9 - R : Conclusions personnelles** (votre réponse ici) :

------
------